In [ ]:
#python 3.13.0
#ag2 0.10.0
#autogen 0.10.0
#chromadb 1.3.4
#google.gnerativeai 0.8.4
#!pip install ag2[gemini]
#!pip install agentops

In [1]:
# from pathlib import Path

In [5]:
# !pip install agentops

In [26]:
import os
# print(os.getenv("GOOGLE_API_KEY"))

In [27]:

import agentops
agentops.init(api_key="c68af40c-9ad7-4c61-a46c-922c98d0f7aa")
agentops.start_session()

🖇 AgentOps: Session Replay for session trace: https://app.agentops.ai/sessions?trace_id=83303570f06a96f95981d83bdb4da361


In [28]:
from pathlib import Path

from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor
work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)

executor = LocalCommandLineCodeExecutor(work_dir=work_dir)

#Agent 1
code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={
        "executor": executor,
    },
    human_input_mode="NEVER",
)

In [29]:
# The code writer agent's system message is to instruct the LLM on how to
# use the Jupyter code executor with IPython kernel.
code_writer_system_message = """
You have been given coding capability to solve tasks using Python code.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, 
    print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the 
    task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your 
language skill.When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other 
action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. 
Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
"""

In [30]:
import os

code_writer_agent = ConversableAgent(
    "code_writer",
    system_message=code_writer_system_message,
    llm_config={"config_list": [{"model": "gemini-2.0-flash", "api_key":os.getenv('GOOGLE_API_KEY'),"api_type": "google"}]},
    code_execution_config=False,  # Turn off code execution for this agent.
    max_consecutive_auto_reply=2,
    human_input_mode="NEVER",)

In [31]:
import pprint

chat_result = code_executor_agent.initiate_chat(
    code_writer_agent, message=" write python code for permutation for the word ALGEbRA. use optimised way to calculate it .I want final result count."
)

pprint.pprint(chat_result)
agentops.end_session("Success")

code_executor_agent (to code_writer):

 write python code for permutation for the word ALGEbRA. use optimised way to calculate it .I want final result count.

--------------------------------------------------------------------------------
code_writer (to code_executor_agent):

```python
# filename: permutation_count.py
from collections import Counter
import math

def count_permutations(word):
  """
  Calculates the number of permutations of a word, taking into account repeated characters.

  Args:
    word: The word to calculate permutations for.

  Returns:
    The number of permutations of the word.
  """
  n = len(word)
  counts = Counter(word.lower())  # Case-insensitive counting
  
  numerator = math.factorial(n)
  denominator = 1
  for count in counts.values():
    denominator *= math.factorial(count)
  
  return numerator // denominator

word = "ALGEbRA"
permutation_count = count_permutations(word)
print(permutation_count)
```


-------------------------------------------------

🖇 AgentOps: Session Replay for session.session trace: https://app.agentops.ai/sessions?trace_id=83303570f06a96f95981d83bdb4da361


In [32]:
# import pprint

# chat_result = code_executor_agent.initiate_chat(
#     code_writer_agent, message="""Write optimized Python code to calculate the number of unique permutations of the word 'ALGEBRA.'
#     For example, the word 'ALGEbRA' should have 2520 unique permutations. Use efficient logic to compute the result and verify the output using 
#     the word 'ALGEBRA.' The code should only return the final permutation count.""")
# pprint.pprint(chat_result)
# agentops.end_session("Success")